In [1]:
##########################################
# This notebook is used for intial data exploration for the capstone project
##########################################


First, we import the data

In [23]:
import pandas as pd
import numpy as np
import glob
from IPython.display import display # Allows the use of display() for DataFrames


data = pd.read_csv('data/transactions_200816.csv')  #import one file

#import all csvs (courtesy of http://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe)

path =r'data' # use your path
allFiles = glob.glob(path + "/tr*.csv")

#print allFiles
#data = pd.DataFrame()
#list_ = []
#for file_ in allFiles:
#    df = pd.read_csv(file_,index_col=None, header=0)
#    list_.append(df)
#data = pd.concat(list_)

print('Before DropNA',len(data))
data=data.dropna()
print('After DropNA',len(data))
#data['CUST_CODE'].head()

#list(data.columns.values)

('Before DropNA', 270664)
('After DropNA', 191470)


Reformat the data into a wide dataset using Prod_code_10 as the lowest level of aggregation.  

In [24]:
#create my target variable
data["CUST_LIFESTAGE_PRICE_SENSITIVITY"] = data["CUST_LIFESTAGE"] + "-" + data["CUST_PRICE_SENSITIVITY"]

#create basketprofiles.  Simplest version will be to sum spend in each category in PROD_CODE_20, keeping the target variable as well
#pivot code from http://stackoverflow.com/questions/41046766/using-and-graphing-the-results-of-a-crosstab-dataframe-in-python

data_cross=data.pivot_table(index='BASKET_ID', columns='PROD_CODE_10', values='SPEND', aggfunc=np.sum, fill_value=0)
data_cross.reset_index(level=['BASKET_ID'], inplace=True)

#create target for the classifier
bybasket=data.groupby(['BASKET_ID'])
targetByBasket=pd.DataFrame(bybasket['CUST_LIFESTAGE_PRICE_SENSITIVITY'].first())
targetByBasket.reset_index(level=['BASKET_ID'], inplace=True)

data_cross = pd.merge(data_cross, targetByBasket, how='inner', on = 'BASKET_ID')

data_cross.head(10)
#list(data_cross.columns.values)

PROD_CODE_10,BASKET_ID,CL00001,CL00002,CL00003,CL00004,CL00005,CL00006,CL00007,CL00008,CL00009,...,CL00242,CL00243,CL00244,CL00245,CL00246,CL00247,CL00248,CL00249,CL00250,CUST_LIFESTAGE_PRICE_SENSITIVITY
0,994111400168267,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,OT-UM
1,994111400168287,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,OT-LA
2,994111400168333,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,OA-UM
3,994111400168387,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,OA-MM
4,994111400168388,0.00,0.0,0.42,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,OA-MM
5,994111400168389,3.09,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,OA-MM
6,994111400168470,0.00,0.0,2.13,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,OA-LA
7,994111400168495,0.00,0.0,0.65,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,OA-UM
8,994111400168506,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,OT-XX
9,994111400168589,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,OA-UM


# Naive Classifier Work

In [ ]:
#do all the work for the naive classifier

#create variables for the naive classifier
spendByBasket=pd.DataFrame(bybasket['SPEND'].sum())
spendByBasket.reset_index(level=['BASKET_ID'], inplace=True)

basketTypeByBasket=pd.DataFrame(bybasket['BASKET_TYPE'].first())
basketTypeByBasket.reset_index(level=['BASKET_ID'], inplace=True)

basketSizeByBasket=pd.DataFrame(bybasket['BASKET_SIZE'].first())
basketSizeByBasket.reset_index(level=['BASKET_ID'], inplace=True)

weekdayByBasket=pd.DataFrame(bybasket['SHOP_WEEKDAY'].first())
weekdayByBasket.reset_index(level=['BASKET_ID'], inplace=True)

#merge them back in
data_cross_naive = pd.merge(data_cross, spendByBasket, how='inner', on = 'BASKET_ID')
data_cross_naive = pd.merge(data_cross_naive, basketTypeByBasket, how='inner', on = 'BASKET_ID')
data_cross_naive = pd.merge(data_cross_naive, basketSizeByBasket, how='inner', on = 'BASKET_ID')
data_cross_naive = pd.merge(data_cross_naive, weekdayByBasket, how='inner', on = 'BASKET_ID')

In [15]:
#create the naive predictor

avgTrainSpend=X_train['SPEND'].mean()
stDevTrainSpend=X_train['SPEND'].std()


def NaiveClassifier(test, trainMean, trainStd):

    if(test['SPEND']<trainMean-trainStd):
            if(test['BASKET_TYPE']='Full Shop')
                   naivePriceSens = 'LA'
            else:
                    naivePriceSens = 'MM'
    elif(test['SPEND']>rainMean-trainStd):
        if(test['BASKET_TYPE']='Full Shop')
                   naivePriceSens = 'MM
            else:
                    naivePriceSens = 'UM'
    else:
        naivePriceSens = 'MM'
    
    if (test["SHOP_WEEKDAY"] = 7) or (test["SHOP_WEEKDAY"] = 1):
        if(test['BASKET_SIZE'] = 'Small'):
            if(test['BASKET_TYPE']='Small Shop')
                   naivePriceSens = 'YA'
            else:
                    naivePriceSens = 'OA'
        elif(test['BASKET_SIZE'] = 'Medium'):
            naiveLifeStage = 'YF'
        else:
            naiveLifeStage = 'OF'            
    else:
        naiveLifeStage = 'PE'
    
    
    
    return naivePriceSens + "-" naiveLifeStage

#apply to every row
#X_test.apply(NaiveClassifier, args=(avgTrainSpend,stDevTrainSpend)) # returns DataFrame


In [ ]:
#pop the two classifiers used for the naive predictor before doing any classification
X_train.pop("SPEND")
X_train.pop("SHOP_WEEKDAY")
X_test.pop("SPEND")
X_test.pop("SHOP_WEEKDAY")

# Split the data

In [25]:
#reset the index to what it should be for the rest
data_cross.set_index(['BASKET_ID'], inplace=True)
#create the training, testing split
from sklearn.cross_validation import train_test_split
#split off the target variable
y = data_cross.pop("CUST_LIFESTAGE_PRICE_SENSITIVITY")
X = data_cross

#split the data into training and testing, from http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42) 

# Dimensionality Reduction

In [30]:
print(X_train.head(10))
from sklearn.decomposition import PCA

pca = PCA(n_components = 6)
pca.fit(X_train)

# TODO: Transform the sample log-data using the PCA fit above
X_train_pca= pca.transform(X_train)
X_test_pca= pca.transform(X_test)

#check cumulative sums
display(np.cumsum(pca.explained_variance_ratio_))


print(X_train_pca)

PROD_CODE_10     CL00001  CL00002  CL00003  CL00004  CL00005  CL00006  \
BASKET_ID                                                               
994111400403246      0.0      0.0      0.0      0.0      0.0      0.0   
994111400493436      0.0      0.0      0.0      0.0      0.0      0.0   
994111400517014      0.0      0.0      0.0      0.0      0.0      0.0   
994111400609586      0.0      0.0      0.0      0.0      0.0      0.0   
994111400683991      0.0      0.0      0.0      0.0      0.0      0.0   
994111400316123      0.0      0.0      0.0      0.0      0.0      0.0   
994111400719413      0.0      0.0      0.0      0.0      0.0      0.0   
994111400809061      0.0      0.0      0.0      0.0      0.0      0.0   
994111400386079      0.0      0.0      0.0      0.0      0.0      0.0   
994111400490319      0.0      0.0      0.0      0.0      0.0      0.0   

PROD_CODE_10     CL00007  CL00008  CL00009  CL00010   ...     CL00241  \
BASKET_ID                                         

array([ 0.98573378,  0.98674812,  0.98768586,  0.9886164 ,  0.98952825,
        0.99014954])

[[ 0.55335525 -0.40599462  0.14857127 -0.12220555 -0.00556478 -0.93562021]
 [ 0.55340434 -0.35498298  0.17349743 -0.12618547 -0.01726352 -0.86178019]
 [ 0.55344136 -0.35067026  0.13536254 -0.09565673 -0.01143364 -0.3869574 ]
 ..., 
 [ 0.553366   -0.3859647   0.14477498 -0.11658124 -0.00377536 -0.80335004]
 [ 0.55343872 -0.3681533   0.14427545 -0.10887298 -0.00578458 -0.47928714]
 [ 0.55335505 -0.39681538  0.15240747 -0.122468   -0.00671079 -0.93248055]]


# try the classifiers

In [41]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators = 100)

rfc = rfc.fit(X_train, y_train)
#rfc_pred = rfc.predict(X_test)
print('non-pca: ', rfc.score(X_test,y_test))

rfc2 = rfc.fit(X_train_pca, y_train)
#rfc_pred = rfc.predict(X_test)
print("pca: ", rfc.score(X_test_pca,y_test))

('non-pca: ', 0.12375043088590142)
('pca: ', 0.099620820406756294)


In [ ]:
#requires sklearn 0.18
#conda update sklearn

from sklearn.neural_network import MLPClassifier
ann = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
ann = ann.fit(X_train, y_train)
ann.score(X_test,y_test)

In [9]:
from sklearn.neighbors import KNeighborsClassifier

nn = KNeighborsClassifier(n_neighbors=1)
nn = nn.fit(X_train,y_train)
#nn_pred = nn.pred(X_test)
nn.score(X_test,y_test)

0.67114788004136505

In [ ]:
from sklearn import tree
dt = tree.DecisionTreeClassifier()
dt = dt.fit(X_train, y_train)
dt.score(X_test,y_test)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier(n_estimators=100)
ada = ada.fit(X_train, y_train)
ada.score(X_test,y_test)

In [ ]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

In [ ]:
#random useful code snippets
list(data.columns.values)
data['SPEND'].dtype
data_cross.head(10)